In [16]:
import time
import numpy as np
import pandas as pd

from tqdm import tqdm


# 데이터 로드

In [21]:
def read_data(route):
    data = pd.read_csv(route)
    data = data[data['ts'] != 'ts']
    data['ts'] = data['ts'].astype('int64')
    # data['time_index'] = ((data['ts'] - data['ts'][0]) / 300) * 5
    
    return data

In [25]:
# 두 데이터 프레임을 ts 기준으로 병합 (하루전가격 기준으로 병합)
기상실측데이터1 = read_data('../OIBC_2024_DATA/data/기상실측데이터_1.csv')
기상실측데이터2 = read_data('../OIBC_2024_DATA/data/기상실측데이터_2.csv')
기상실측데이터3 = read_data('../OIBC_2024_DATA/data/기상실측데이터_3.csv')
기상실측데이터 = pd.concat([기상실측데이터1, 기상실측데이터2, 기상실측데이터3], ignore_index=True)

# location에 대한 one-hot encoding
# ['Ilgwa-ri' 'Geumak-ri' 'Yongsu-ri' 'Hacheon-ri'
# 'Cheonji-dong' 'Songdang-ri' 'Bonggae-dong' 'Gwangryeong-ri' 'Sangmo-ri']
locations = 기상실측데이터['location'].unique()
기상실측데이터 = pd.get_dummies(기상실측데이터, columns=['location'])
기상실측데이터 = 기상실측데이터.fillna(0)

기상실측데이터

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Cheonji-dong,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri
0,1709222220,6.9,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,False,False,True,False,False,False,False
1,1709222580,7.1,2.7,2.7,68.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,False,False,True,False,False,False,False
2,1709226120,6.3,0.9,0.9,64.0,-0.1,338.0,22.0,22.0,0.0,...,False,False,False,False,False,True,False,False,False,False
3,1709229420,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,...,False,False,False,False,False,True,False,False,False,False
4,1709229840,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106075,1731155514,18.96,0,0,0,0,90.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
106076,1731157267,18.08,0,0,0,0,100.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
106077,1731159045,18.96,0,0,0,0,90.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
106078,1731160878,17.91,0,0,0,0,104.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False


In [26]:
실시간가격1 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_실시간가격.csv')
실시간가격2 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_실시간가격_2.csv')
실시간가격 = pd.concat([실시간가격1, 실시간가격2], ignore_index=True)
# 하루전가격 = 하루전가격.drop(0)  # 첫 데이터는 학습 데이터가 없어서 제거
실시간가격

,ts,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh)
0,1709218800,95.30,95.30
1,1709219700,170.95,170.95
2,1709220600,170.42,170.42
3,1709221500,107.39,107.39
4,1709222400,107.39,107.39
...,...,...,...
24090,1730988000,111.71,111.71
24091,1730988900,104.64,104.64
24092,1730989800,104.64,104.64
24093,1730990700,104.64,104.64


# 데이터 병합

In [27]:
data = pd.merge(기상실측데이터, 실시간가격, on="ts", how="outer")

# location에 따른 interpolation
# for location in locatio합location_{location}'] == True]
#     rows = rows.interpolate(method='linear')
#     # print(type(rows), rows[0])
#     data = pd.concat([data, rows], ignore_index=True)
    
# nan 가진 데이터 제거 
# data = data.dropna(axis=0)

# ts를 기준으로 sort
data = data.sort_values('ts')
data.reset_index(inplace=True)
data = data.drop(columns=['index'])

data

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh)
0,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.3,95.3
1,1709218850,3.23,0,0,0,0,343.0,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
2,1709218851,7.18,0,0,0,0,341.0,0,0,0,...,False,False,False,False,True,False,False,False,NaN,NaN
3,1709218856,8.03,0,0,0,0,352.0,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
4,1709218980,8.5,8.0,8.0,81.0,5.5,270.0,6.1,17.6,0.0,...,False,False,False,False,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130106,1731162646,14.96,0,0,0,0,90.0,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
130107,1731162647,18.96,0,0,0,0,90.0,0,0,0,...,False,False,False,False,True,False,False,False,NaN,NaN
130108,1731162649,19.03,0,0,0,0,90.0,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
130109,1731164160,18.6,16.6,16.6,70.0,13.0,315.0,10.8,21.7,0.0,...,False,False,False,False,False,False,False,True,NaN,NaN


# Interpolation Class

In [28]:
def sort_values_and_remove_index(data):
    new_data = data.copy()
    new_data = new_data.sort_values('ts')
    new_data.reset_index(inplace=True)
    new_data.drop(columns=['index'], inplace=True)
    
    return new_data


class InterpolationHelper:
    def __init__(self, data, column_names, data_filter=None):
        self.data = data
        self.column_names = column_names
        self.value_finder = data_filter
        
        # 만약 특정 Location 같은 곳 안에서만 필터링 하고 싶으면 적용함
        if self.value_finder is not None:
            self.data = self.value_finder(data).copy()
            self.data.reset_index(inplace=True)
            self.data.drop(columns=['index'], inplace=True)
            
        for column_name in self.column_names:
            self.data[column_name] = self.data[column_name].astype('float64')
            
    def calculate_interpolation_by_value(self, index1, index2, index3, column_name):
        # 결측치 또는 불완전한 값은 필터링
        if index1 is None and index3 is None:
            return None
        if index1 is None:
            return self.data.iloc[index3, :][column_name]
        if index3 is None:
            return self.data.iloc[index1, :][column_name]
        
        time1 = self.data.iloc[index1, :]['ts']
        time2 = self.data.iloc[index2, :]['ts']
        time3 = self.data.iloc[index3, :]['ts']
        
        column1 = self.data.iloc[index1, :][column_name]
        column2 = self.data.iloc[index2, :][column_name]
        column3 = self.data.iloc[index3, :][column_name]
        
        # print(column1, column2, column3, time1, time2, time3)
        
        # nan 아니면 기존 값 사용
        if pd.notna(column2):
            return column2
    
        if time3 > time1:
            diff = (time2 - time1) / (time3 - time1)
            return column1 + diff * (column3 - column1)
        else:
            return column1
        
    def find_value_notna(self, index, column_name):
        # 앞쪽에서 가장 가까운 NaN이 아닌 값의 인덱스 찾기
        previous_non_nan_index = self.data[column_name][:index].last_valid_index()
    
        # 뒤쪽에서 가장 가까운 NaN이 아닌 값의 인덱스 찾기
        if index + 1 >= len(self.data):
            return previous_non_nan_index, None
        
        next_non_nan_index = self.data[column_name][index + 1:].first_valid_index()
        
        return previous_non_nan_index, next_non_nan_index
        
    def calculate_interpolation(self):
        new_data = self.data.copy()
        new_data = sort_values_and_remove_index(new_data)
        
        for index, row in tqdm(new_data.iterrows(), ncols=75, total=len(new_data)):
            for column_name in self.column_names:
                if pd.notna(row[column_name]):
                    continue
                previous_index, next_index = self.find_value_notna(index, column_name)
                next_value = self.calculate_interpolation_by_value(previous_index, index, next_index, column_name)
                new_data.loc[index, column_name] = next_value
            
            
        return new_data

# interpolation_helper_krw = InterpolationHelper(data, '하루전가격(원/kWh)', lambda x: x['location_Bonggae-dong'])

# 각 location에 대한 새 데이터 생성

In [29]:
interpolation_data = data.copy()

price_data = interpolation_data[pd.notna(interpolation_data['실시간 확정 가격(원/kWh)'])]

# 한 가격에 대해 모든 지역에서의 데이터를 각각 생성합니다
for location in locations:
    data_inner = price_data.copy()
    for inner_location in locations:
        data_inner[f'location_{inner_location}'] = False
    
    data_inner[f'location_{location}'] = True
    interpolation_data = pd.concat([interpolation_data, data_inner], ignore_index=True)
    
interpolation_data.dropna(subset=[f'location_{locations[0]}'], inplace=True)
interpolation_data = sort_values_and_remove_index(interpolation_data)
interpolation_data

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh)
0,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,False,False,95.3,95.3
1,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,True,False,False,False,95.3,95.3
2,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,False,False,False,False,False,95.3,95.3
3,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,True,95.3,95.3
4,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,95.3,95.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399919,1731162646,14.96,0,0,0,0,90.0,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
399920,1731162647,18.96,0,0,0,0,90.0,0,0,0,...,False,False,False,False,True,False,False,False,NaN,NaN
399921,1731162649,19.03,0,0,0,0,90.0,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
399922,1731164160,18.9,16.8,16.8,63.0,12.2,90.0,22.2,22.2,0.0,...,False,False,False,False,False,False,False,False,NaN,NaN


# 데이터 Interpolation (가격제외)

In [30]:
columns = [
    'temp', 'real_feel_temp','real_feel_temp_shade','rel_hum','dew_point',
    'wind_dir','wind_spd','wind_gust_spd','uv_idx','vis','cld_cvr','ceiling',
    'pressure','appr_temp','wind_chill_temp','wet_bulb_temp','precip_1h',
    'cloud','temp_max','temp_min','humidity','ground_press','wind_speed',
    'rain','snow'
]
print('columns:', len(columns), columns)
print('locations:', len(locations), locations)

new_interpolation_data = None
for location in locations:
    interpolation_helper_loc = InterpolationHelper(
        interpolation_data, 
        columns, 
        lambda x: x[x[f'location_{location}'] == True]
    )
    new_data = interpolation_helper_loc.calculate_interpolation()
    
    if new_interpolation_data is None:
        new_interpolation_data = new_data
    else:
        new_interpolation_data = pd.concat([new_interpolation_data, new_data], ignore_index=True)
            
    # new_interpolation_data.to_csv('test.csv', index=False)
            
new_interpolation_data = sort_values_and_remove_index(new_interpolation_data)
new_interpolation_data

columns: 25 ['temp', 'real_feel_temp', 'real_feel_temp_shade', 'rel_hum', 'dew_point', 'wind_dir', 'wind_spd', 'wind_gust_spd', 'uv_idx', 'vis', 'cld_cvr', 'ceiling', 'pressure', 'appr_temp', 'wind_chill_temp', 'wet_bulb_temp', 'precip_1h', 'cloud', 'temp_max', 'temp_min', 'humidity', 'ground_press', 'wind_speed', 'rain', 'snow']
locations: 12 ['Ilgwa-ri' 'Geumak-ri' 'Yongsu-ri' 'Hacheon-ri' 'Cheonji-dong'
 'Songdang-ri' 'Bonggae-dong' 'Gwangryeong-ri' 'Sangmo-ri' 'Gaigeturi'
 'Cheju-do' 'Jeju']


100%|███████████████████████████████| 36690/36690 [04:09<00:00, 147.32it/s]


,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh)
0,1709218800,6.90,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,True,False,False,False,False,95.3,95.3
1,1709218800,8.50,8.0,8.0,81.0,5.5,270.0,6.1,17.6,0.0,...,False,False,False,False,False,False,False,False,95.3,95.3
2,1709218800,7.70,0.0,0.0,67.0,1.9,270.0,40.8,43.2,0.0,...,False,False,False,False,False,False,False,True,95.3,95.3
3,1709218800,5.00,2.6,2.6,86.0,2.8,293.0,11.2,15.5,0.0,...,False,False,False,False,False,False,True,False,95.3,95.3
4,1709218800,7.80,5.7,5.7,65.0,2.2,338.0,11.1,11.1,0.0,...,False,False,False,False,False,False,False,False,95.3,95.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399919,1731162646,14.96,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,NaN,NaN
399920,1731162647,18.96,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,...,False,False,False,False,True,False,False,False,NaN,NaN
399921,1731162649,19.03,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,NaN,NaN
399922,1731164160,18.60,16.6,16.6,70.0,13.0,315.0,10.8,21.7,0.0,...,False,False,False,False,False,False,False,True,NaN,NaN


# 데이터 Interpolation (가격)

In [31]:
original_price_list = pd.notna(new_interpolation_data['실시간 확정 가격(원/kWh)'])
new_interpolation_data['확정가격여부'] = original_price_list

interpolation_helper = InterpolationHelper(
    new_interpolation_data, 
    ['실시간 확정 가격(원/kWh)']
)
new_interpolation_data = interpolation_helper.calculate_interpolation()

new_interpolation_data

100%|████████████████████████████| 399924/399924 [01:36<00:00, 4158.31it/s]


,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh),확정가격여부
0,1709218800,6.90,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,True,False,False,False,False,95.3,95.30,True
1,1709218800,6.10,1.1,1.1,80.0,2.9,158.0,18.0,18.0,0.0,...,False,True,False,False,False,False,False,95.3,95.30,True
2,1709218800,3.23,0.0,0.0,0.0,0.0,343.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,95.3,95.30,True
3,1709218800,8.03,0.0,0.0,0.0,0.0,352.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,95.3,95.30,True
4,1709218800,4.90,-1.8,-1.8,66.0,-1.0,293.0,26.5,31.8,0.0,...,False,False,False,False,False,False,False,95.3,95.30,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399919,1731162646,14.96,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,NaN,104.64,False
399920,1731162647,18.96,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,NaN,104.64,False
399921,1731162649,19.03,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,NaN,104.64,False
399922,1731164160,18.60,16.6,16.6,70.0,13.0,315.0,10.8,21.7,0.0,...,False,False,False,False,False,False,True,NaN,104.64,False


# 결측치 제거

In [32]:
data_cleaned = new_interpolation_data.dropna(axis=0)
data_cleaned

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh),확정가격여부
0,1709218800,6.900000,2.40,2.40,69.0,1.5,338.000000,18.7,37.7,0.0,...,False,False,True,False,False,False,False,95.30,95.30,True
1,1709218800,6.100000,1.10,1.10,80.0,2.9,158.000000,18.0,18.0,0.0,...,False,True,False,False,False,False,False,95.30,95.30,True
2,1709218800,3.230000,0.00,0.00,0.0,0.0,343.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,95.30,95.30,True
3,1709218800,8.030000,0.00,0.00,0.0,0.0,352.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,95.30,95.30,True
4,1709218800,4.900000,-1.80,-1.80,66.0,-1.0,293.000000,26.5,31.8,0.0,...,False,False,False,False,False,False,False,95.30,95.30,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399103,1730991600,19.420000,18.26,18.26,50.0,8.6,85.800000,9.7,9.7,0.0,...,False,False,False,False,False,False,False,104.64,104.64,True
399104,1730991600,15.480000,12.82,12.82,60.8,7.9,68.000000,19.5,19.5,0.0,...,False,False,False,False,False,False,True,104.64,104.64,True
399105,1730991600,15.060000,11.84,11.84,80.0,11.6,0.000000,23.4,23.4,0.0,...,False,False,False,False,True,False,False,104.64,104.64,True
399106,1730991600,10.808398,0.00,0.00,0.0,0.0,36.596685,0.0,0.0,0.0,...,False,False,False,False,False,False,False,104.64,104.64,True


# 데이터 저장하기

In [33]:
new_interpolation_data.to_csv('train_dataset.csv', index=False)
data_cleaned.to_csv('train_dataset_clean.csv', index=False)

# 가격 데이터가 존재하는 컬럼만 추출

In [35]:
only_has_price = data_cleaned[data_cleaned['확정가격여부'] == True]
only_has_price

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh),확정가격여부
0,1709218800,6.900000,2.40,2.40,69.0,1.5,338.000000,18.7,37.7,0.0,...,False,False,True,False,False,False,False,95.30,95.30,True
1,1709218800,6.100000,1.10,1.10,80.0,2.9,158.000000,18.0,18.0,0.0,...,False,True,False,False,False,False,False,95.30,95.30,True
2,1709218800,3.230000,0.00,0.00,0.0,0.0,343.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,95.30,95.30,True
3,1709218800,8.030000,0.00,0.00,0.0,0.0,352.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,95.30,95.30,True
4,1709218800,4.900000,-1.80,-1.80,66.0,-1.0,293.000000,26.5,31.8,0.0,...,False,False,False,False,False,False,False,95.30,95.30,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399103,1730991600,19.420000,18.26,18.26,50.0,8.6,85.800000,9.7,9.7,0.0,...,False,False,False,False,False,False,False,104.64,104.64,True
399104,1730991600,15.480000,12.82,12.82,60.8,7.9,68.000000,19.5,19.5,0.0,...,False,False,False,False,False,False,True,104.64,104.64,True
399105,1730991600,15.060000,11.84,11.84,80.0,11.6,0.000000,23.4,23.4,0.0,...,False,False,False,False,True,False,False,104.64,104.64,True
399106,1730991600,10.808398,0.00,0.00,0.0,0.0,36.596685,0.0,0.0,0.0,...,False,False,False,False,False,False,False,104.64,104.64,True


In [36]:
only_has_price.to_csv('train_dataset_price_only.csv', index=False)

# 현황데이터도 merge

In [37]:
conditions1 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_현황데이터.csv')
conditions2 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_현황데이터_2.csv')
conditions = pd.concat([conditions1, conditions2], ignore_index=True)
conditions

,ts,공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
0,1704034800,1298000.0,769000.0,0.0,125426.0,133103.0,529000.0,323000.0
1,1704035100,1299000.0,761000.0,0.0,130393.0,137867.0,538000.0,339000.0
2,1704035400,1290000.0,755000.0,0.0,120691.0,128133.0,534000.0,335000.0
3,1704035700,1277000.0,756000.0,0.0,107771.0,115519.0,521000.0,325000.0
4,1704036000,1269000.0,753000.0,0.0,99264.4,107153.0,515000.0,318000.0
...,...,...,...,...,...,...,...,...
89223,1731162900,1103000.0,608000.0,0.0,27479.2,27997.4,495000.0,202000.0
89224,1731163200,1100000.0,604000.0,0.0,24471.4,24990.9,496000.0,206000.0
89225,1731163500,1101000.0,601000.0,0.0,25811.0,26331.1,499000.0,211000.0
89226,1731163800,1107000.0,599000.0,0.0,30547.1,31066.2,508000.0,218000.0


In [38]:
data_with_conditions = pd.merge(only_has_price, conditions, on='ts', how='inner')
data_with_conditions

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh),확정가격여부,공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
0,1709218800,6.900000,2.40,2.40,69.0,1.5,338.000000,18.7,37.7,0.0,...,95.30,95.30,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
1,1709218800,6.100000,1.10,1.10,80.0,2.9,158.000000,18.0,18.0,0.0,...,95.30,95.30,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
2,1709218800,3.230000,0.00,0.00,0.0,0.0,343.000000,0.0,0.0,0.0,...,95.30,95.30,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
3,1709218800,8.030000,0.00,0.00,0.0,0.0,352.000000,0.0,0.0,0.0,...,95.30,95.30,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
4,1709218800,4.900000,-1.80,-1.80,66.0,-1.0,293.000000,26.5,31.8,0.0,...,95.30,95.30,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290703,1730991600,19.420000,18.26,18.26,50.0,8.6,85.800000,9.7,9.7,0.0,...,104.64,104.64,True,1097000.0,605000.0,0.0,62513.4,62977.1,492000.0,326000.0
290704,1730991600,15.480000,12.82,12.82,60.8,7.9,68.000000,19.5,19.5,0.0,...,104.64,104.64,True,1097000.0,605000.0,0.0,62513.4,62977.1,492000.0,326000.0
290705,1730991600,15.060000,11.84,11.84,80.0,11.6,0.000000,23.4,23.4,0.0,...,104.64,104.64,True,1097000.0,605000.0,0.0,62513.4,62977.1,492000.0,326000.0
290706,1730991600,10.808398,0.00,0.00,0.0,0.0,36.596685,0.0,0.0,0.0,...,104.64,104.64,True,1097000.0,605000.0,0.0,62513.4,62977.1,492000.0,326000.0


In [39]:
data_with_conditions.to_csv('train_dataset_with_condition.csv', index=False)

# 하루전 가격 + 실시간 가격

In [8]:
하루전가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_하루전가격.csv')
실시간가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_실시간가격.csv')

실시간가격 = 실시간가격.drop(columns=['실시간 임시 가격(원/kWh)'])
하루전_실시간 = pd.merge(하루전가격, 실시간가격, on='ts', how='inner')

하루전_실시간.to_csv('days.csv', index=False)